# Regression on biodiversity index

We are going to test different models on our dataset, trying to get a better results using a grid search and the testing the models on a dataset of a different region.

For each model we're going to use a RandomizedSearchCV to narrow our parameters research and the the GridSearchCV to find the best one.

In [79]:
import pandas as pd
import numpy as np
import glob
import os
import sys
import torch
from itertools import combinations 

from scipy.stats import uniform, randint
from sklearn.decomposition import PCA

from sklearn import preprocessing

from sklearn.pipeline import make_pipeline, Pipeline

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor 
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor

import xgboost as xgb

module_this = os.path.abspath(os.path.join(os.getcwd()))
modules = [ module_this]

for module in modules:
    if module not in sys.path:
        sys.path.append(module)

import utils as ut

In [15]:
def report_best_scores(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

## Load Datasets

In [36]:
folder = "../Dataset"
regression_label = 'habitat_richness'
test_size = 0.2
swi_labels = ['SWI1km-SWI-002', 'SWI1km-SWI-100', 'SWI1km-SWI-040', 'SWI1km-SWI-005', 
              'SWI1km-SWI-010', 'SWI1km-SWI-060', 'SWI1km-SWI-015', 'SWI1km-SWI-020']

datas = []

paths = [f for f in glob.glob(folder + "/*.csv") if 'closest_point_mean' in f and 'finland' not in f]
paths += [f for f in glob.glob(folder + "/*.csv") if 'remove' in f and 'finland' in f]
for path in paths:
    df = pd.read_csv(path, index_col=['longitude', 'latitude'])

    if(df.isna().any().any()):
        print(path, '\t has ', df.isna().any().sum(), ' row with null values')
    
    y = df[regression_label].values
    X = df.drop(columns=[regression_label]).values
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42, shuffle=True)
    datas.append({'name': path[11:-4], 'dataframe': df, 'y': y, 'X': X, 
                'X_train': X_train, 'X_test': X_test, 
            'y_train': y_train, 'y_test': y_test})

In [37]:
[print(x['name'], "\t--\t",x['dataframe'].shape) for x in datas]

bulgaria_out_closest_point_mean_handle_custom_set 	--	 (14458, 47)
france_out_closest_point_mean_handle_custom_set 	--	 (1882, 47)
italy_out_closest_point_mean_handle_custom_set 	--	 (17387, 47)
finland_out_remove_handle_set_null 	--	 (17654, 47)


[None, None, None, None]

## Random Forest Regressor

In [38]:
cv = KFold(n_splits=5, shuffle=True, random_state = 42)
scaler = preprocessing.MinMaxScaler()

Testing datasets with all available features

In [34]:
for data in datas:
    rfr = RandomForestRegressor(random_state = 42)
    model = make_pipeline(scaler, rfr)
    val_score = cross_val_score(model, data['X_train'], data['y_train'], cv=cv, n_jobs=-1, verbose=0)
    data['val_score'] = val_score
    print(data['name'], "\t validation score: \t", "{:.3f}".format(val_score.mean()), " +/- ", "{:.3f}".format(val_score.std()))
    model.fit(data['X_train'], data['y_train'])
    #print(model.score(data['X_train'], data['y_train']))
    test_score = model.score(data['X_test'], data['y_test'])
    print(data['name'], "\t test score: \t\t", "{:.3f}".format(test_score))
    print('--------------------------------------------')

finland_out_closest_point_mean_handle_custom_set 	 validation score: 	 0.717  +/-  0.003
finland_out_closest_point_mean_handle_custom_set 	 test score: 		 0.712
--------------------------------------------
finland_out_knn_handle_custom_set 	 validation score: 	 0.716  +/-  0.003
finland_out_knn_handle_custom_set 	 test score: 		 0.712
--------------------------------------------
finland_out_mean_handle_custom_set 	 validation score: 	 0.717  +/-  0.003
finland_out_mean_handle_custom_set 	 test score: 		 0.711
--------------------------------------------
finland_out_remove_handle_set_null 	 validation score: 	 0.711  +/-  0.008
finland_out_remove_handle_set_null 	 test score: 		 0.720
--------------------------------------------


Parameters for RandomSearch

In [9]:
n_estimators = [50, 100, 200, 500, 1000]
max_features = ['auto', 'sqrt']
max_depth = [10, 20, 50, 100, None]
max_depth = [50]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

grid_params = {'rfr__n_estimators': n_estimators,
               'rfr__max_features': max_features,
               'rfr__max_depth': max_depth,
               'rfr__min_samples_split': min_samples_split,
               'rfr__min_samples_leaf': min_samples_leaf,
               'rfr__bootstrap': bootstrap}
grid_params

{'rfr__n_estimators': [50, 100, 200, 500, 1000],
 'rfr__max_features': ['auto', 'sqrt'],
 'rfr__max_depth': [50, None],
 'rfr__min_samples_split': [2, 5, 10],
 'rfr__min_samples_leaf': [1, 2, 4],
 'rfr__bootstrap': [True, False]}

In [ ]:

for data in datas:
    rfr = RandomForestRegressor()
    model = Pipeline([('scaler', scaler), ('rfr', rfr)])

    grid_search = GridSearchCV(estimator = model, param_grid = grid_params, 
                              cv = cv, n_jobs = -1, verbose = 1)
    grid_search.fit(data['X_train'], data['y_train'])
    print(data['name'], ' cross validation best score: \t',  "{:.3f}".format(grid_search.best_score_))
    torch.save(grid_search.best_params_, data['name'] + "__bset_params")
    data['rfr_best'] = grid_search.best_params_
    
    best_model = grid_search.best_estimator_
    best_model.fit(data['X_train'], data['y_train'])
    test_score = best_model.score(data['X_test'], data['y_test'])
    print(data['name'], "\t best model test score: \t\t", "{:.3f}".format(test_score))
    print('--------------------------------------------')
    

Fitting 5 folds for each of 360 candidates, totalling 1800 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 25.6min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 123.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 168.7min
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed: 325.9min


## Working with all data

In [76]:
X_all = np.vstack((datas[0]['X'], datas[1]['X'], datas[2]['X'], datas[3]['X']))
y_all = np.concatenate((datas[0]['y'], datas[1]['y'], datas[2]['y'], datas[3]['y']))

region_id = np.hstack(np.array([[i] * d['X'].shape[0] for i, d in enumerate(datas)]))

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size=0.2, stratify=region_id, random_state=42)


In [77]:
cv = KFold(n_splits=5, shuffle=True, random_state = 42)
scaler = preprocessing.MinMaxScaler()

In [78]:
rfr = RandomForestRegressor(random_state = 42)
model = make_pipeline(scaler, rfr)
val_score = cross_val_score(model, X_train, y_train, cv=cv, n_jobs=-1, verbose=0)
print("All data validation score: \t", "{:.3f}".format(val_score.mean()), " +/- ", "{:.3f}".format(val_score.std()))
model.fit(X_train, y_train)
test_score = model.score(X_test, y_test)
print(data['name'], "\t test score: \t\t", "{:.3f}".format(test_score))
print('--------------------------------------------')

All data validation score: 	 0.962  +/-  0.001
finland_out_remove_handle_set_null 	 test score: 		 -0.070
--------------------------------------------


### Leave one region out

In [114]:
reg_idx = [0,1,2,3]
comb = combinations(reg_idx, 3)

for c in list(comb):
    X_train = []
    y_train = []
    for i in c:
        X_train.append(datas[i]['X'])
        y_train.append(datas[i]['y'])
        
    X_train = np.vstack(X_train)
    y_train = np.hstack(y_train)
    
    test_idx = list(set(reg_idx) - set(c))[0]
   
    X_test = datas[test_idx]['X']
    y_test = datas[test_idx]['y']
    
    rfr = RandomForestRegressor(random_state = 42)
    model = make_pipeline(scaler, rfr)
    model.fit(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print("Prediction of", datas[test_idx]['name'], "score: ", "{:.3f}".format(test_score))
    print('------------------------------')
    
    

Prediction of finland_out_remove_handle_set_null score:  -26.166
------------------------------
Prediction of italy_out_closest_point_mean_handle_custom_set score:  -5.524
------------------------------
Prediction of france_out_closest_point_mean_handle_custom_set score:  -0.908
------------------------------
Prediction of bulgaria_out_closest_point_mean_handle_custom_set score:  -2.060
------------------------------
